# **Gestor de expedientes clinicos y analizador de salud**

## ***Data4Life***

**. :Daniel Núñez**

**. :Angelo Bonifacio**

**. :Isaac Herrera**

**. :Marco Reyes**

# **Importando librerias**

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from google.colab import output
import seaborn as sns
import os
import time

# **Definiendo funciones**

In [3]:
df = pd.DataFrame()

In [4]:
# Funcion para cargar dataset existente
def cargar_ds(archivo):
  try:
    df = pd.read_csv(archivo)
    return df

  except FileNotFoundError:
    print("El archivo no existe")
  else:
    return df


In [5]:
# Funcion para agregar un paciente al dataset
def agregar_paciente(df):
    nombre = input("Nombre: ")
    edad = int(input("Edad: "))
    genero = input("Género (Masculino/Femenino): ")
    altura = float(input("Altura (cm): "))
    peso = float(input("Peso (kg): "))
    sistolica = int(input("Presión Sistólica: "))
    diastolica = int(input("Presión Diastólica: "))
    colesterol = int(input("Colesterol: "))
    glucosa = int(input("Glucosa: "))

    # Crear diccionario con los datos del nuevo paciente
    nuevo_paciente = {
        'Nombre': nombre,
        'Edad': edad,
        'Genero': genero,
        'Altura_cm': altura,
        'Peso_kg': peso,
        'Presion_Sistolica': sistolica,
        'Presion_Diastolica': diastolica,
        'Colesterol_mg_dL': colesterol,
        'Glucosa_mg_dL': glucosa
    }

    # Convertir a DataFrame y concatenar
    nuevo_df = pd.DataFrame([nuevo_paciente])
    df_actualizado = pd.concat([df, nuevo_df], ignore_index=True)

    return df_actualizado

In [6]:
# Funcion para buscar paciente por su indice o id en el dataset
def buscar_paciente(df, indice):
    paciente = df.loc[indice]
    print(paciente)

In [7]:
def guardar_ds(df):
  df.to_csv('Data_clinica.csv')

In [8]:
df = cargar_ds('Data_clinica.csv')

El archivo no existe


In [9]:
# Funcion para eliminar paciente del dataset
def eliminar_paciente(df, indice):
  df.drop(indice, inplace=True)

In [10]:
# ----------------- 1. Métricas antropométricas -----------------
def calcular_imc(peso_kg, altura_cm):
    altura_m = altura_cm / 100
    return peso_kg / (altura_m ** 2)

def categoria_imc(imc):
    if imc < 18.5: return 'Bajo peso'
    elif imc < 25: return 'Normal'
    elif imc < 30: return 'Sobrepeso'
    elif imc < 35: return 'Obesidad I'
    elif imc < 40: return 'Obesidad II'
    else: return 'Obesidad III'

# -Presión arterial
def calcular_pam(ps, pd):
    return (2*pd + ps) / 3

def categoria_presion(ps, pd):
    if ps < 90 or pd < 60:
        return 'Hipotensión'
    elif ps < 120 and pd < 80:
        return 'Normal'
    elif 120 <= ps < 130 and pd < 80:
        return 'Elevada'
    elif 130 <= ps < 140 or 80 <= pd < 90:
        return 'Hipertensión I'
    elif 140 <= ps <= 180 or 90 <= pd <= 120:
        return 'Hipertensión II'
    else:
        return 'Crisis hipertensiva'

#  Colesterol
def categoria_colesterol(col):
    if col < 200: return 'Óptimo'
    elif col < 240: return 'Límite alto'
    else: return 'Alto'

#  Glucosa
def categoria_glucosa(glu):
    if glu < 100: return 'Normal'
    elif glu < 126: return 'Prediabetes'
    else: return 'Diabetes'

#  Índice de Riesgo Cardiometabólico
def normalizar(valor, minimo, maximo):
    return (valor - minimo) / (maximo - minimo)

def calcular_irc(pam, col, glu):
    norm_pam = normalizar(pam, 70, 110)
    norm_col = normalizar(col, 150, 300)
    norm_glu = normalizar(glu, 70, 160)
    return np.mean([norm_pam, norm_col, norm_glu])

def categoria_irc(irc):
    if irc < 0.4: return 'Bajo'
    elif irc < 0.7: return 'Moderado'
    else: return 'Alto'

#  6. Índice de Salud Global
def calcular_isg(pam, col, glu, imc):
    D_PAM = abs((pam - 90)/90)
    D_Col = abs((col - 180)/180)
    D_Glu = abs((glu - 90)/90)
    D_IMC = abs((imc - 22)/22)
    α, β, γ, δ = 0.25, 0.25, 0.25, 0.25
    return 100 - 100*(α*D_PAM + β*D_Col + γ*D_Glu + δ*D_IMC)

def categoria_isg(isg):
    if isg >= 85: return 'Excelente'
    elif isg >= 70: return 'Aceptable'
    elif isg >= 50: return 'Riesgo moderado'
    else: return 'Riesgo alto'

# Ajuste demográfico
def factor_edad(edad):
    if edad < 30: return 0.0
    elif edad < 50: return 0.1
    else: return 0.25

def factor_genero(genero, edad):
    genero = genero.lower()
    if genero.startswith('m'):
        return 1.1 if edad < 50 else 1.0
    else:
        return 1.0 if edad < 50 else 1.1

#  Función principal para aplicar todas las métricas
def calcular_metricas_modular(df):

    df = df.copy()
    df['IMC'] = df.apply(lambda x: calcular_imc(x['Peso_kg'], x['Altura_cm']), axis=1)
    df['Categoria_IMC'] = df['IMC'].apply(categoria_imc)

    df['PAM'] = df.apply(lambda x: calcular_pam(x['Presion_Sistolica'], x['Presion_Diastolica']), axis=1)
    df['Categoria_Presion'] = df.apply(lambda x: categoria_presion(x['Presion_Sistolica'], x['Presion_Diastolica']), axis=1)

    df['Categoria_Colesterol'] = df['Colesterol_mg_dL'].apply(categoria_colesterol)
    df['Categoria_Glucosa'] = df['Glucosa_mg_dL'].apply(categoria_glucosa)

    df['IRC'] = df.apply(lambda x: calcular_irc(x['PAM'], x['Colesterol_mg_dL'], x['Glucosa_mg_dL']), axis=1)
    df['Categoria_IRC'] = df['IRC'].apply(categoria_irc)

    df['ISG'] = df.apply(lambda x: calcular_isg(x['PAM'], x['Colesterol_mg_dL'], x['Glucosa_mg_dL'], x['IMC']), axis=1)
    df['Categoria_ISG'] = df['ISG'].apply(categoria_isg)

    df['Factor_Edad'] = df['Edad'].apply(factor_edad)
    df['Factor_Genero'] = df.apply(lambda x: factor_genero(x['Genero'], x['Edad']), axis=1)

    return df

## **Visualizacion de datos**

In [11]:
def visualizacion_graficas(df):
  df = df.copy()

  # Configuración del estilo
  plt.style.use('default')
  sns.set_palette("husl")

  # Crear figura con subgráficos
  fig = plt.figure(figsize=(20, 15))

  # 1. Distribución por Género
  plt.subplot(3, 4, 1)
  df['Genero'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['lightblue', 'lightpink'])
  plt.title('Distribución por Género')
  plt.ylabel('')

  # 2. Distribución de Edad
  plt.subplot(3, 4, 2)
  plt.hist(df['Edad'], bins=15, color='skyblue', edgecolor='black', alpha=0.7)
  plt.title('Distribución de Edad')
  plt.xlabel('Edad (años)')
  plt.ylabel('Frecuencia')

  # 3. Distribución de Altura
  plt.subplot(3, 4, 3)
  plt.hist(df['Altura_cm'], bins=15, color='lightgreen', edgecolor='black', alpha=0.7)
  plt.title('Distribución de Altura')
  plt.xlabel('Altura (cm)')
  plt.ylabel('Frecuencia')

  # 4. Distribución de Peso
  plt.subplot(3, 4, 4)
  plt.hist(df['Peso_kg'], bins=15, color='orange', edgecolor='black', alpha=0.7)
  plt.title('Distribución de Peso')
  plt.xlabel('Peso (kg)')
  plt.ylabel('Frecuencia')

  # 5. Distribución del IMC
  plt.subplot(3, 4, 5)
  plt.hist(df['IMC'], bins=20, color='coral', edgecolor='black', alpha=0.7)
  plt.axvline(df['IMC'].mean(), color='red', linestyle='--', label=f'Media: {df["IMC"].mean():.1f}')
  plt.title('Distribución del IMC')
  plt.xlabel('Índice de Masa Corporal')
  plt.ylabel('Frecuencia')
  plt.legend()

  # 6. Categorías de IMC
  plt.subplot(3, 4, 6)
  categorias_imc = df['Categoria_IMC'].value_counts()
  plt.bar(categorias_imc.index, categorias_imc.values, color=['green', 'yellow', 'orange', 'red'])
  plt.title('Categorías de IMC')
  plt.xticks(rotation=45)
  plt.ylabel('Frecuencia')

  # 7. Categorías de Presión Arterial
  plt.subplot(3, 4, 7)
  categorias_presion = df['Categoria_Presion'].value_counts()
  plt.bar(categorias_presion.index, categorias_presion.values, color=['green', 'lightblue', 'orange', 'red'])
  plt.title('Categorías de Presión Arterial')
  plt.xticks(rotation=45)
  plt.ylabel('Frecuencia')

  # 8. Distribución de Colesterol
  plt.subplot(3, 4, 8)
  plt.hist(df['Colesterol_mg_dL'], bins=15, color='brown', edgecolor='black', alpha=0.7)
  plt.title('Distribución de Colesterol')
  plt.xlabel('Colesterol (mg/dL)')
  plt.ylabel('Frecuencia')

  # 9. Categorías de Colesterol
  plt.subplot(3, 4, 9)
  categorias_colesterol = df['Categoria_Colesterol'].value_counts()
  plt.bar(categorias_colesterol.index, categorias_colesterol.values, color=['green', 'yellow', 'red'])
  plt.title('Categorías de Colesterol')
  plt.xticks(rotation=45)
  plt.ylabel('Frecuencia')

  # 10. Distribución de Glucosa
  plt.subplot(3, 4, 10)
  plt.hist(df['Glucosa_mg_dL'], bins=15, color='teal', edgecolor='black', alpha=0.7)
  plt.title('Distribución de Glucosa')
  plt.xlabel('Glucosa (mg/dL)')
  plt.ylabel('Frecuencia')

  # 11. IMC por Género
  plt.subplot(3, 4, 11)
  sns.boxplot(data=df, x='Genero', y='IMC')
  plt.title('IMC por Género')
  plt.xlabel('Género')
  plt.ylabel('IMC')

  plt.tight_layout()
  plt.show()

  # 12. Comparación de Categorías de Riesgo (gráfico separado)
  plt.figure(figsize=(12, 6))
  categorias_riesgo = pd.DataFrame({
      'Presión': df['Categoria_Presion'].value_counts(),
      'Colesterol': df['Categoria_Colesterol'].value_counts(),
      'Glucosa': df['Categoria_Glucosa'].value_counts()
  })
  categorias_riesgo.plot(kind='bar')
  plt.title('Comparación de Categorías de Riesgo')
  plt.xlabel('Categorías')
  plt.ylabel('Frecuencia')
  plt.legend(title='Variables de Riesgo')
  plt.xticks(rotation=45)
  plt.grid(axis='y', alpha=0.3)
  plt.tight_layout()
  plt.show()

# **Interfaz**

In [13]:
# Interfaz
def main():

  df = cargar_ds('Data_clinica.csv') # Se carga el archivo csv
  df = calcular_metricas_modular(df) # Se calculan las metricas todos los pacientes
  print('Cargando archivo...')
  time.sleep(2)

  while True:



    output.clear()
    print("""

          a) Agregar paciente
          b) Buscar paciente
          c) Eliminar paciente
          d) metricas de todos los pacientes
          e) Visualizar graficas
          f) salir
  """)

    opcion = input("Ingrese una opción: ")

    match opcion:
      case 'a':
        output.clear()
        df = agregar_paciente(df)
        df = calcular_metricas_modular(df)
        input('Presione enter tecla para continuar...')

      case 'b':
        output.clear()
        indice = int(input("Ingrese el indice del paciente: "))
        buscar_paciente(df, indice)
        input('Presione enter tecla para continuar...')

      case 'c':
        output.clear()
        indice = int(input("Ingrese el indice del paciente: "))
        eliminar_paciente(df, indice)
        input('Presione enter tecla para continuar...')

      case 'd':
        output.clear()
        print(df)
        input('Presione enter tecla para continuar...')

      case 'e':
        visualizacion_graficas(df)
        input('Presione enter tecla para continuar...')

      case 'f':
        guardar_ds(df)
        print('Saliendo...')
        time.sleep(2)
        break

main()





          a) Agregar paciente
          b) Buscar paciente
          c) Eliminar paciente
          d) metricas de todos los pacientes
          e) Visualizar graficas
          f) salir
  
Ingrese una opción: f
Saliendo...
